In [ ]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(device)

cuda


In [ ]:
transformer = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ColorJitter(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                          [0.229, 0.224, 0.225]) 
])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_path="/content/drive/MyDrive/BrainDead_Combined/Train"
test_path="/content/drive/MyDrive/BrainDead_Combined/Test"
 
train_loader = DataLoader(
     torchvision.datasets.ImageFolder(train_path,transform = transformer),
     batch_size = 128, shuffle = True
 )

 
test_loader = DataLoader(
     torchvision.datasets.ImageFolder(test_path,transform = transformer),
     batch_size = 128, shuffle = True
 )

In [ ]:
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1]for j in root.iterdir()])

In [ ]:
print(classes)

['angry', 'happy', 'neutral', 'sad']


In [ ]:
class ConvNet(nn.Module):
  def __init__(self, num_classes = 5):
    super(ConvNet,self).__init__()

    #First Conv block
    self.conv1 = nn.Conv2d(in_channels=3,out_channels=16, kernel_size=3, stride=1,padding=1)
    self.bn1 = nn.BatchNorm2d(num_features=16)
    self.relu1 = nn.ReLU()
    self.pool1 = nn.MaxPool2d(kernel_size=2)

    #Second Conv Block
    self.conv2 = nn.Conv2d(in_channels=16,out_channels=32, kernel_size=3, stride=1,padding=1)
    self.bn2 = nn.BatchNorm2d(num_features=32)
    self.relu2 = nn.ReLU()
    self.pool2 = nn.MaxPool2d(kernel_size=2)

    self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1,padding=1)
    self.bn3 = nn.BatchNorm2d(num_features=64)
    self.relu3 = nn.ReLU()
    self.pool3 = nn.MaxPool2d(kernel_size=2)

    self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=1, stride=1,padding=1)
    self.bn4 = nn.BatchNorm2d(num_features=128)
    self.relu4 = nn.ReLU()
    self.pool4 = nn.MaxPool2d(kernel_size=2)

    self.fc=nn.Linear(in_features = 128*17*17 ,out_features=256)

    self.fc1=nn.Linear(in_features = 256 ,out_features=num_classes)

  def forward(self,input):
    output=self.conv1(input)
    output=self.bn1(output)
    output=self.relu1(output)
    output=self.pool1(output)

    output=self.conv2(output)
    output=self.bn2(output)
    output=self.relu2(output)
    output=self.pool2(output)
    
    output=self.conv3(output)
    output=self.bn3(output)
    output=self.relu3(output)
    output=self.pool3(output)

    output=self.conv4(output)
    output=self.bn4(output)
    output=self.relu4(output)
    output=self.pool4(output)

    output=output.view(-1,128*17*17)

    output=self.fc(output)
    output=self.fc1(output)
    return output

In [ ]:
model = ConvNet(num_classes=4).to(device)

In [ ]:
optimizer = Adam(model.parameters(), lr=2*0.00001, weight_decay=0.0001)
loss_function = nn.CrossEntropyLoss()

In [ ]:
num_epochs = 100

In [ ]:
train_count = len(glob.glob(train_path+'/**/*.png'))
test_count = len(glob.glob(test_path+'/**/*.png'))

In [ ]:
print(train_count, test_count)

2289 589


In [ ]:
from tqdm import tqdm as tqdm

best_accuracy = 0.0

for epoch in tqdm(range(num_epochs)) :

  model.train()
  train_accuracy=0.0
  train_loss = 0.0

  for i, (images,labels) in enumerate(train_loader):  
    if torch.cuda.is_available():
      images= Variable(images.cuda())
      labels = Variable(labels.cuda())

    optimizer.zero_grad()
    outputs = model(images)
    loss = loss_function(outputs,labels)
    loss.backward()
    optimizer.step()

    train_loss+=loss.cpu().data*images.size(0)
    _,prediction = torch.max(outputs.data,1)

    train_accuracy+=int(torch.sum(prediction==labels.data))

  train_accuracy = train_accuracy/train_count
  train_loss = train_loss/train_count

  model.eval()

  test_accuracy=0.0
  for i, (images,labels) in enumerate(test_loader):  
    if torch.cuda.is_available():
      images= Variable(images.cuda())
      labels = Variable(labels.cuda())

    outputs = model(images)
    _,prediction=torch.max(outputs.data,1)
    test_accuracy+=int(torch.sum(prediction==labels.data))

  test_accuracy = test_accuracy/test_count
  print("\n")
  print('Epoch: '+str(epoch)+' Train Loss: '+str(int(train_loss))+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
  if test_accuracy>best_accuracy:
    torch.save(model.state_dict(),'best_checkpoint.model')
    best_accuracy = test_accuracy


  1%|          | 1/100 [02:15<3:43:56, 135.72s/it]



Epoch: 0 Train Loss: 1 Train Accuracy: 0.35561380515508956 Test Accuracy: 0.28183361629881154


  2%|▏         | 2/100 [02:43<1:58:05, 72.30s/it] 



Epoch: 1 Train Loss: 1 Train Accuracy: 0.48623853211009177 Test Accuracy: 0.3938879456706282


  3%|▎         | 3/100 [03:11<1:24:19, 52.16s/it]

...



Epoch: 97 Train Loss: 0 Train Accuracy: 0.9899519440803844 Test Accuracy: 0.7096774193548387


 99%|█████████▉| 99/100 [47:23<00:27, 27.55s/it]



Epoch: 98 Train Loss: 0 Train Accuracy: 0.9908256880733946 Test Accuracy: 0.7164685908319185


100%|██████████| 100/100 [47:50<00:00, 28.71s/it]



Epoch: 99 Train Loss: 0 Train Accuracy: 0.9934469200524246 Test Accuracy: 0.7079796264855688


Epoch: 99 Train Loss: 0 Train Accuracy: 0.9823204419889503 Test Accuracy: 0.
5745257452574526 on Original Data


Epoch: 99 Train Loss: 0 Train Accuracy: 1.0 Test Accuracy: 0.590909090909091 on Synthetic only


Epoch: 95 Train Loss: 0 Train Accuracy: 0.9908256880733946 Test Accuracy: 0.7181663837011885
 97%|█████████▋| 97/100 [46:28<01:22, 27.62s/it]


In [ ]:
model.to('cuda')

ConvNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu4): ReL